# 子词切分方法的压缩率比较

## 实验目的
基于课程网站上的英文数据，利用子词切分方法对英文数据进行切分，分别设置词表规模为1000、3000和5000，统计计算切分后的数据压缩率。

## 实验环境
- Python 3.12.9
- Miniconda

## 实验原理

### WordPiece

WordPiece是一种基于频率的子词分割方法，其核心思想是通过在训练过程中逐渐构建一个子词词表，以最大化词汇的似然性。它最初由Google提出并应用于BERT模型中。其通过贪心算法不断选择频率最高的子词对进行合并，直到满足设定的词表大小。具体步骤为：

1. 从一个字母级别的标记（字符）开始，构建一个初步的子词词表。
2. 在每轮迭代中，选择最频繁的字符对进行合并，直到词表达到预定的大小。
3. 子词分割基于该词表进行，遇到未在词表中的词汇时，进行进一步的分割。

### BPE（Byte Pair Encoding）

BPE是一种基于字符对频率的分割方法，通过反复合并频率最高的字符对来构建一个新的子词。其最初是为了进行文本压缩提出的，但后来广泛应用于自然语言处理中的子词切分。具体步骤为：

1. 初始化字符级词汇表，每个词汇都是一个独立的字符。
2. 统计所有字符对的频率。
3. 合并频率最高的字符对，形成新的词汇。
4. 重复此过程，直到达到预定的词表大小。

与WordPiece不同，BPE不考虑上下文，而是仅依赖于字符对的出现频率，因此其分割方式相对简单。

### Unigram

Unigram模型是一种基于概率的子词分割方法，假设每个子词都是独立的，并通过最大化数据的似然估计来选择子词切分。其通常使用一种基于期望最大化（EM）算法的方法来生成子词词表。具体步骤为：

1. 初始时，子词词表包含所有单词。
2. 使用EM算法，通过迭代计算每个子词的概率，以最大化数据集的对数似然。
3. 在每次迭代中，评估和更新子词的概率分布，直到收敛到预定的词表大小。

Unigram模型较为复杂，相比BPE和WordPiece，它更多地依赖于数据的概率分布进行分割。

### 压缩率

压缩率是衡量数据压缩效果的重要指标，定义为压缩前的大小与压缩后的大小的比值：

\[
\text{压缩率} = \frac{\text{压缩前的文件大小}}{\text{压缩后的文件大小}}
\]

在子词切分方法中，压缩率用于衡量通过子词切分后，数据存储所需的空间变化。切分后的文本将通过较小的子词词汇表进行表示，因此能有效减少存储空间。

## 实验过程

### 分析数据集

数据集名为`news-commentary-v6`，包含……五种语言的文本数据。打印数据集：

In [ ]:
from pathlib import Path
DATA_DIR = Path("./data")
DATASET_NAME = 'news-commentary-v6'
lang = 'en'

with open(DATA_DIR / f'{DATASET_NAME}.{lang}') as f:
    text = f.read()

print(text[:200])

可以看到数据组织为每行一句话，我们可以将整个文件作为一整个字符串进行分词器的训练和分词。

### 训练分词器


In [ ]:
from itertools import product
from tokenizers import Tokenizer
from tokenizers.models import WordPiece, BPE, Unigram
from tokenizers.trainers import WordPieceTrainer, BpeTrainer, UnigramTrainer
from tokenizers.pre_tokenizers import Whitespace

VOCAB_SIZE = [1000, 3000, 5000]
NAME_TEMPLATE = "{model}-{vocab_size}.json"
TOKENIZER_DIR = Path("./tokenizers")
if not TOKENIZER_DIR.exists():
    TOKENIZER_DIR.mkdir()
models = [WordPiece, BPE, Unigram]
trainers = [WordPieceTrainer, BpeTrainer, UnigramTrainer]


for vocab_size, (model, trainer) in product(VOCAB_SIZE, zip(models, trainers)):
    print(f"Training {model.__name__} with vocab size {vocab_size}")
    tokenizer = Tokenizer(model())
    tokenizer.pre_tokenizer = Whitespace()
    trainer = trainer(vocab_size=vocab_size, special_tokens=["[UNK]"])
    tokenizer.train([str(DATA_DIR / f'{DATASET_NAME}.{lang}')], trainer)
    file_name = NAME_TEMPLATE.format(model=model.__name__, vocab_size=vocab_size)
    tokenizer.save(str(TOKENIZER_DIR / file_name))